<a href="https://colab.research.google.com/github/GuihDM-dev/car-color-recognition/blob/main/notebooks/CarColorRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Vamos começar instalando e importando oque será necessario


In [1]:
# Instalar as dependências
!pip install ultralytics opencv-python matplotlib gdown


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 13.7 MB/s eta 0:00:00


In [2]:
#Importanto o dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("landrykezebou/vcor-vehicle-color-recognition-dataset")

print("Path to dataset files:", path)

100%|██████████| 574M/574M [00:05<00:00, 108MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1


##Verificando o dataset e adptando o para o modelo Yolo


In [3]:
import os

# Verificar a estrutura do dataset
dataset_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1"
for root, dirs, files in os.walk(dataset_path):
    print(f"Root: {root}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files[:5]}")  # Mostrar apenas os 5 primeiros arquivos

Root: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1
Dirs: ['test', 'train', 'val']
Files: []
Root: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/test
Dirs: ['orange', 'brown', 'red', 'purple', 'silver', 'tan', 'yellow', 'beige', 'gold', 'green', 'black', 'white', 'blue', 'grey', 'pink']
Files: []
Root: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/test/orange
Dirs: []
Files: ['ebf177406e.jpg', 'c6c43da016.jpg', '0b538b22be.jpg', '5f7d080bfb.jpg', '1104c21047.jpg']
Root: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/test/brown
Dirs: []
Files: ['378a5f4fd6.jpg', '5f39cea59f.jpg', '584364d3fe.jpg', '1d949385cb.jpg', '82b520a874.jpg']
Root: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/test/red
Dirs: []
Files: ['a362535ff6.jpg', '6bfe720ee2.jpg

In [4]:
import os

def load_dataset(base_path):
    data = []
    for label in os.listdir(base_path):
        label_path = os.path.join(base_path, label)
        if os.path.isdir(label_path):
            for file in os.listdir(label_path):
                if file.endswith('jpg'):
                    file_path = os.path.join(label_path, file)
                    data.append((file_path, label))
    return data

#base path appra o conjunto de treino
train_base_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train"
train_data = load_dataset(train_base_path)

print(train_data[:5])

val_base_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/val"
val_data = load_dataset(val_base_path)

print(val_data[:5])

[('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/8abc24b776.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/bf3338124b.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/431ca69ab7.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/cb393cfa12.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/9c9e374970.jpg', 'orange')]
[('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/val/orange/788108ae2f.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/val/orange/f6aaf06153.jpg', 'orange'), ('/root/.cache/kagglehub/data

In [5]:
color_mapping = {
    "purple": 0, "blue": 1, "green": 2, "red": 3, "tan": 4,
    "white": 5, "grey": 6, "orange": 7, "beige": 8, "silver": 9,
    "yellow": 10, "brown": 11, "black": 12, "pink": 13, "gold": 14
}

In [6]:
import os
import shutil
from PIL import ImageChops, Image

#Diretorios de origem
# Diretório raiz do dataset
dataset_dir = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1"

# Inicializar lista de dados
data = []

# Iterar pelas pastas de treino e validação
for split in ['train', 'val', 'test']:
    split_dir = os.path.join(dataset_dir, split)
    for label in os.listdir(split_dir):  # Cada subpasta é uma classe
        label_dir = os.path.join(split_dir, label)
        for image_name in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_name)
            data.append((image_path, label))

# Exibir os primeiros itens para validação
print(data[:10])

#classes e mapeamento de cores
classes = ['purple', 'beige', 'tan', 'pink', 'gold', 'silver',  'blue', 'red', 'yellow', 'white', 'black', 'grey', 'green', 'orange', 'brown']
class_to_id = {cls: idx for idx, cls in enumerate(classes)}

#diretorios de destino
output_dir = "yolo_dataset"
image_dir = os.path.join(output_dir, "images")
label_dir = os.path.join(output_dir, "labels")

train_img_dir = os.path.join(image_dir, "train")
val_img_dir = os.path.join(image_dir, "val")
train_label_dir = os.path.join(label_dir, "train")
val_label_dir = os.path.join(label_dir, "val")
test_img_dir = os.path.join(image_dir, "test")
test_label_dir = os.path.join(label_dir, "test")

#criação das pastas de cada um
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(test_img_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

#Função que transformar anotações em formato YOLO
def create_yolo_label(image_path, label, label_dir):
    #abrir imagem para pegpar as dimensões
    with Image.open(image_path) as img:
        width, height = img.size

    # YOLO espera coordenadas normalizadas (aqui como classificação, centro é 0.5,0.5 e dimensão total)
    class_id = class_to_id[label]
    center_x, center_y = 0.5, 0.5
    obj_width, obj_height = 1.0, 1.0  # Imagem inteira, pois é uma classificação

    # Escrever o arquivo de anotação
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(label_dir, f"{image_name}.txt")
    with open(label_path, "w") as f:
        f.write(f"{class_id} {center_x} {center_y} {obj_width} {obj_height}\n")

# Separar dados entre treino, validação e teste
for image_path, label in data:
    if "train" in image_path:
        dest_img_dir = train_img_dir
        dest_label_dir = train_label_dir
    elif "val" in image_path:
        dest_img_dir = val_img_dir
        dest_label_dir = val_label_dir
    elif "test" in image_path:
        dest_img_dir = test_img_dir
        dest_label_dir = test_label_dir
    else:
        continue  # Ignorar se não for treino, validação ou teste

    # Copiar imagem e criar anotação
    dest_path = os.path.join(dest_img_dir, os.path.basename(image_path))
    shutil.copy(image_path, dest_path)
    create_yolo_label(image_path, label, dest_label_dir)

# Criar o arquivo data.yaml
data_yaml_path = os.path.join(output_dir, "data.yaml")
with open(data_yaml_path, "w") as f:
    f.write(f"train: {os.path.abspath(train_img_dir)}\n")
    f.write(f"val: {os.path.abspath(val_img_dir)}\n")
    f.write(f"test: {os.path.abspath(test_img_dir)}\n")
    f.write("nc: {}\n".format(len(classes)))
    f.write(f"names: {classes}\n")

print(f"Conversão para YOLOv8 concluída. Dados salvos em: {output_dir}")





[('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/8abc24b776.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/bf3338124b.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/431ca69ab7.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/cb393cfa12.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/9c9e374970.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/7faa08efdd.jpg', 'orange'), ('/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1/train/orange/b25debdc55.jpg', 'orange'), ('/root/.cache/kagglehub/d